<a href="https://colab.research.google.com/github/JYEmm-eng/Final-Team9/blob/main/9%ED%8C%80_%ED%8C%8C%EC%9D%B4%EB%84%90_%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TMDB Data 전처리 과정

In [ ]:
#revenue, budget = null, 0인 행 필터링(파일 크기 이슈로 VSCODE 통해 진행)

import pandas as pd

file_path = 'TMDB_movie_dataset_v11.csv'
# 새로 저장할 파일의 이름을 정합니다.
output_file_path = 'filteredTMDB.csv'

# 첫 번째 덩어리인지 확인하기 위한 변수를 만듭니다. (헤더를 한 번만 저장하기 위함)
is_first_chunk = True

# 파일을 10만 행씩 잘라서 순차적으로 처리
chunk_iterator = pd.read_csv(
    file_path,
    chunksize=100000,
    encoding='utf-8',
    low_memory=False
)

print(f"필터링을 시작합니다. 결과는 '{output_file_path}' 파일에 저장됩니다...")

# 각 덩어리(chunk)를 순회하며 조건에 맞는 행을 찾고 저장합니다.
for i, chunk in enumerate(chunk_iterator):
    chunk['budget'] = pd.to_numeric(chunk['budget'], errors='coerce').fillna(0)
    chunk['revenue'] = pd.to_numeric(chunk['revenue'], errors='coerce').fillna(0)

    # --- 핵심 조건 ---
    valid_rows = chunk[(chunk['budget'] != 0)]

    # 찾은 데이터가 있을 경우에만 저장 작업을 수행합니다.
    if not valid_rows.empty:
        if is_first_chunk:
            # 첫 번째 덩어리는 헤더와 함께 새 파일로 저장합니다.
            valid_rows.to_csv(
                output_file_path,
                index=False,
                encoding='utf-8-sig'
            )
            is_first_chunk = False  # 다음부터는 이 코드를 실행하지 않도록 변경
        else:
            # 두 번째 덩어리부터는 헤더 없이 기존 파일에 내용을 추가합니다.
            valid_rows.to_csv(
                output_file_path,
                mode='a',
                header=False,
                index=False,
                encoding='utf-8-sig'
            )

    print(f"{i+1}번째 덩어리 처리 중...")

print("\n--- 작업 완료 ---")
print(f"'{output_file_path}' 파일이 생성되었습니다.")

In [ ]:
#TMDB API 크롤링(VSCODE 통해 진행)

import pandas as pd
import requests
import time
from tqdm import tqdm

# --- 1. API 키 설정 ---
# TMDB에서 발급받은 본인의 API 키를 여기에 입력하세요.
API_KEY = 'API_KEY' # 예: 'a1b2c3d4e5f6a1b2c3d4e5f6a1b2c3d4'

def get_credits(movie_id):
    """
    주어진 movie_id로 감독(Director)과 상위 5명의 배우(Cast) 이름을 찾아 반환하는 함수
    """
    time.sleep(0.05) # API 서버에 부담을 주지 않기 위해 잠시 대기
    url = f"https://api.themoviedb.org/3/movie/{int(movie_id)}/credits?api_key={API_KEY}&language=en-US"

    director = None
    cast = None

    try:
        response = requests.get(url, timeout=10)

        if response.status_code == 200:
            data = response.json()

            # 감독 찾기
            for member in data.get('crew', []):
                if member.get('job') == 'Director':
                    director = member.get('name')
                    break # 첫 번째 감독만 찾고 종료

            # 상위 5명 배우 이름 가져오기
            cast_list = [actor.get('name') for actor in data.get('cast', [])[:5]]
            if cast_list:
                cast = ', '.join(cast_list) # 쉼표로 구분된 텍스트로 변환

        # 찾은 감독과 배우 목록을 반환
        return director, cast

    except requests.exceptions.RequestException:
        # 연결 오류 시 둘 다 에러로 처리
        return "Connection Error", "Connection Error"

def main():
    """메인 실행 함수"""
    if API_KEY == 'YOUR_API_KEY':
        print("\n[오류] 코드의 API_KEY 부분을 본인의 TMDB API 키로 변경한 후 다시 실행해주세요.")
        return

    print("TMDB API를 사용하여 감독 및 배우 정보를 가져오는 작업을 시작합니다.")

    try:
        df = pd.read_csv('FILE_PATH', encoding='utf-8')
        print(f"\n'filteredTMDB.csv' 파일을 성공적으로 읽었습니다. (총 {len(df)}개 행)")
    except FileNotFoundError:
        print("\n[오류] 'filteredTMDB.csv' 파일을 찾을 수 없습니다.")
        return

    # 각 영화 ID에 대해 get_credits 함수를 적용하고 결과를 저장할 리스트
    results = []

    # tqdm을 사용하여 진행 상황 표시
    for movie_id in tqdm(df['id'], desc="감독/배우 정보 가져오는 중"):
        results.append(get_credits(movie_id))

    # 결과를 새로운 컬럼으로 데이터프레임에 추가
    df[['director', 'cast']] = results

    # --- 결과 저장 ---
    output_filename = 'TMDB_with_credits.csv'
    df.to_csv(output_filename, index=False, encoding='utf-8-sig')

    print("\n--- 작업 완료 ---")
    print(f"결과가 '{output_filename}' 파일에 저장되었습니다.")

if __name__ == '__main__':
    main()

**데이터 전처리**

- 조건 1: 'backdrop_path', 'homepage', 'poster_path' 컬럼 삭제
- 조건 2: 'vote_count'가 0인 행 삭제
- 조건 3: 'vote_count'가 1이고 'keywords'에 'wrestling'이 포함된 행 삭제
- 조건 4: 'genres', 'production_countries', 'director', 'keywords' 컬럼이 null인 행 삭제
- 조건 5: Budget 30만 달러 미만 행 삭제
- ROI 및 SR 계산 로직 추가
- 흥행척도(y_result) 컬럼 추가

In [ ]:
import pandas as pd
import os
import numpy as np

# 1. 파일 경로 설정
# 코랩에 직접 파일을 업로드한 경우 이 경로를 사용합니다.
file_path = '/content/TMDB_with_credits.csv'

# 2. CSV 파일 불러오기
try:
    print(f"Loading data from: {file_path}")
    df = pd.read_csv(file_path)
    print("Data loaded successfully. Initial shape:", df.shape)
    print("\nInitial DataFrame info:")
    df.info()
except FileNotFoundError:
    print(f"Error: The file was not found at the specified path: {file_path}")
    print("Please make sure you have uploaded 'TMDB_with_credits.csv' to your Colab session.")
    # 파일이 없을 경우, 이후 코드 실행을 막기 위해 빈 데이터프레임 생성
    df = pd.DataFrame()

# 3. 데이터 가공 (파일이 성공적으로 로드된 경우에만 실행)
if not df.empty:
    # 복사본을 만들어 원본 데이터 보존
    df_processed = df.copy()

    # 조건 1: 'backdrop_path', 'homepage', 'poster_path' 컬럼 삭제
    columns_to_drop = ['backdrop_path', 'homepage', 'poster_path']
    # 해당 컬럼이 데이터프레임에 존재하는지 확인 후 삭제
    existing_columns_to_drop = [col for col in columns_to_drop if col in df_processed.columns]
    if existing_columns_to_drop:
        df_processed.drop(columns=existing_columns_to_drop, inplace=True)
        print(f"\nDropped columns: {existing_columns_to_drop}. Shape after dropping columns:", df_processed.shape)
    else:
        print("\nColumns to drop not found in the DataFrame.")


    # 조건 2: 'vote_count'가 0인 행 삭제
    initial_rows = df_processed.shape[0]
    df_processed = df_processed[df_processed['vote_count'] != 0]
    print(f"\nRemoved rows with vote_count = 0. Shape after filtering:", df_processed.shape)
    print(f"{initial_rows - df_processed.shape[0]} rows were removed.")

    # 조건 3: 'vote_count'가 1이고 'keywords'에 'wrestling'이 포함된 행 삭제
    initial_rows = df_processed.shape[0]
    # 'keywords' 컬럼이 문자열이 아닌 경우(NaN 등) 오류가 발생하지 않도록 astype(str)으로 변환하고,
    # 'wrestling' 문자열을 포함하는지 확인합니다. na=False는 NaN 값을 False로 처리합니다.
    condition_to_remove = (df_processed['vote_count'] == 1) & (df_processed['keywords'].astype(str).str.contains('wrestling', na=False))
    # 위 조건에 해당하지 않는(~) 행들만 선택하여 데이터프레임을 갱신합니다.
    df_processed = df_processed[~condition_to_remove]
    print(f"\nRemoved rows with vote_count = 1 and 'wrestling' in keywords. Shape after filtering:", df_processed.shape)
    print(f"{initial_rows - df_processed.shape[0]} rows were removed.")

    # 조건 4: 'genres', 'production_countries', 'director', 'keywords' 컬럼이 null인 행 삭제
    columns_to_check_null = ['genres', 'production_countries', 'director', 'keywords']
    initial_rows = df_processed.shape[0]
    # 해당 컬럼들이 존재하는지 확인
    existing_columns_to_check = [col for col in columns_to_check_null if col in df_processed.columns]
    if existing_columns_to_check:
        df_processed.dropna(subset=existing_columns_to_check, inplace=True)
        print(f"\nRemoved rows with null values in {existing_columns_to_check}. Final shape:", df_processed.shape)
        print(f"{initial_rows - df_processed.shape[0]} rows were removed.")
    else:
        print("\nColumns to check for nulls not found in the DataFrame.")

    # 조건 5: Budget 30만 달러 미만 행 삭제
    df_processed = df_processed[df_processed['budget'] >= 300000]

    # --- ROI 및 SR 계산 로직 추가 ---

    # 1. ROI(수익률) 계산 컬럼 추가
    df_processed['ROI'] = ((df_processed['revenue'] - df_processed['budget']) / df_processed['budget']) * 100

    # 2. SR(흥행 등급) 계산 컬럼 추가
    conditions = [
        (df_processed['revenue'] / df_processed['budget']) >= 3,
        (df_processed['revenue'] / df_processed['budget']) >= 2.5,
        (df_processed['revenue'] / df_processed['budget']) >= 2,
    ]
    choices = [3, 2, 1]
    df_processed['SR'] = np.select(conditions, choices, default=0)

    # 3. y_result 컬럼 추가
    df_processed['y_result'] = 0

    # ROI 조건
    df_processed['y_result'] = df_processed['y_result'] + np.where(df_processed['ROI'] > 2.5, 1, 0)

    # SR 조건
    df_processed['y_result'] = df_processed['y_result'] + np.where(df_processed['SR'] > 2, 1, 0)

    # 예산 및 매출 조건
    commercial_cond = df_processed['budget'] <= 100000000
    blockbuster_cond = df_processed['budget'] > 100000000

    df_processed['y_result'] = df_processed['y_result'] + np.where(
        commercial_cond & (df_processed['revenue'] >= 100000000), 1, 0)
    df_processed['y_result'] = df_processed['y_result'] + np.where(
        blockbuster_cond & (df_processed['revenue'] >= 500000000), 1, 0)


    # 4. 가공된 데이터 확인
    print("\n--- Processed Data Sample ---")
    print(df_processed.head())

    # 5. 가공된 파일을 새로운 CSV로 저장
    # 원본 파일이 있던 디렉토리에 저장합니다.
    output_directory = os.path.dirname(file_path) if os.path.dirname(file_path) else '/content/'
    output_filename = 'TMDB_processed_final.csv'
    output_path = os.path.join(output_directory, output_filename)

    try:
        df_processed.to_csv(output_path, index=False, encoding='utf-8-sig')
        print(f"\nProcessed data successfully saved to: {output_path}")
    except Exception as e:
        print(f"\nAn error occurred while saving the file: {e}")

else:
    print("\nSkipping data processing due to file loading error.")

# 가설 1
원작, 실화 기반 영화의 흥행력이 높을 것이다.

In [ ]:
import pandas as pd
import numpy as np

# 데이터 불러오기
df = pd.read_csv('./TMDB_processed_final.csv')
keyword_columns = ['keywords']

# 'based on' 포함 여부를 나타내는 마스크 생성
mask = df[keyword_columns].astype(str).apply(lambda x: x.str.contains('based on', case=False, na=False)).any(axis=1)

# 'is_based_on_original' 열 추가
# 원작 기반이면 1, 아니면 0으로 구분합니다.
df['is_based_on_original'] = np.where(mask, 1, 0)

# 두 그룹의 영화 수 확인
print(df['is_based_on_original'].value_counts())

# 원작 기반 여부와 흥행 결과(y_result) 간의 교차표 생성
contingency_table = pd.crosstab(df['is_based_on_original'], df['y_result'])

print("--- 교차표 (빈도) ---")
print(contingency_table)

contingency_table_ratio = pd.crosstab(df['is_based_on_original'], df['y_result'], normalize='index')

print("\n--- 교차표 (그룹 내 비율) ---")
print(contingency_table_ratio)

# 시각화하여 확인
import matplotlib.pyplot as plt
import seaborn as sns

contingency_table_ratio.plot(kind='bar', stacked=True)
plt.title('Proportion of Box Office Success by Movie Type')
plt.xlabel('Movie Type (0: Original, 1: Based on Original)')
plt.ylabel('Proportion')
plt.xticks(rotation=0)
plt.legend(title='y_result')
plt.savefig('success_proportion_by_type.png')
plt.show()


## 카이제곱 검정

from scipy.stats import chi2_contingency

# 2단계에서 만든 교차표를 카이제곱 검정에 사용
chi2, p_value, dof, expected = chi2_contingency(contingency_table)

print(f"카이제곱 통계량: {chi2:.4f}")
print(f"P-value: {p_value}")

# p-value를 기준으로 가설을 판단합니다.
alpha = 0.05  # 유의수준 5%
if p_value < alpha:
    print(f"\nP-value ({p_value:.4f})가 유의수준 ({alpha})보다 작으므로, 귀무가설을 기각합니다.")
    print(">> 결론: 원작 기반 여부와 흥행 결과는 통계적으로 유의미한 관련이 있습니다.")
else:
    print(f"\nP-value ({p_value:.4f})가 유의수준 ({alpha})보다 크므로, 귀무가설을 기각할 수 없습니다.")
    print(">> 결론: 원작 기반 여부와 흥행 결과가 관련이 있다는 통계적 근거를 찾지 못했습니다.")

# 가설 2
과거에 흥행한 이력이 있는 감독은 차기작도 흥행할 것이다.

In [ ]:
## 추가 전처리

# 연도순 정렬 (release_date 컬럼이 날짜형인지 확인 후 변환)
df1['release_date'] = pd.to_datetime(df1['release_date'])
df1 = df1.sort_values(by=['director', 'release_date']).reset_index(drop=True)

# 첫 번째 칼럼: 감독이 이전에 찍은 영화의 개수
df1['di_count'] = (
    df1.groupby('director')
       .cumcount()
)

# 두 번째 칼럼: 감독이 이전에 찍은 영화 중 성공(y_result>=2) 개수
df1['sucessed'] = (
    df1.groupby('director')['y_result']
       .apply(lambda x: (x >= 2).cumsum().shift(fill_value=0))
       .reset_index(drop=True)
)

# 세 번째 칼럼: ratio = sucessed / di_count (0 division 방지)
df1['ratio'] = df1.apply(
    lambda row: row['sucessed'] / row['di_count'] if row['di_count'] > 0 else 0,
    axis=1
)

df1.tail(200)

df1.to_csv("tmdb_profinal_di_counts.csv", index=False)


## t-test

# ratio 기준으로 두 그룹 나누기 (중위수/평균/임계치 기준 선택 가능)
threshold = df1['ratio'].median()  # 여기서는 median(중위수) 기준으로 나눔

group_high = df1[df1['ratio'] > threshold]['y_result']
group_low = df1[df1['ratio'] <= threshold]['y_result']

# 독립표본 t-test
t_stat, p_value = stats.ttest_ind(group_high, group_low, equal_var=False)  # Welch’s t-test

print("고 ratio 그룹(양수) y_result 평균:", group_high.mean())
print("저 ratio 그룹(음수) y_result 평균:", group_low.mean())
print("t-통계량:", t_stat)
print("p-값:", p_value)

if p_value < 0.05:
    print("✅ 유의수준 0.05에서 두 그룹 간 차이가 통계적으로 유의합니다.")
else:
    print("❌ 유의한 차이를 찾지 못했습니다.")


## 아노바 검정

# df1이 있다고 가정
df = df1.copy()

# 세 구간 정의
group1 = df[(df['ratio'] >= 0.0) & (df['ratio'] < 0.2)]['y_result']   # [0,0.2)
group2 = df[(df['ratio'] >= 0.2) & (df['ratio'] < 0.7)]['y_result']   # [0.2,0.7)
group3 = df[(df['ratio'] >= 0.7) & (df['ratio'] < 1.0)]['y_result']  # [0.7,1]

# 샘플 수 확인
print("샘플 수:")
print("[0,0.2):", len(group1))
print("[0.2,0.7):", len(group2))
print("[0.7,1]:", len(group3))
print()

# 일원분산분석 (One-way ANOVA)
f_stat, p_value = stats.f_oneway(group1, group2, group3)

print("ANOVA F-통계량:", f_stat)
print("ANOVA p-value :", p_value)

# 각 그룹 평균 출력
print("\n그룹 평균 y_result: ratio 구간이 높아질수록, 즉 감독의 전작 성공률이 높아질수 평균 y_result 상승")
print("[0,0.2):", group1.mean())
print("[0.2,0.7):", group2.mean())
print("[0.7,1]:", group3.mean())


## 시각화

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# 데이터프레임 df 에서 그룹 나누기
group1 = df1[(df1['ratio'] >= 0.0) & (df1['ratio'] < 0.2)]['y_result']
group2 = df1[(df1['ratio'] >= 0.2) & (df1['ratio'] < 0.7)]['y_result']
group3 = df1[(df1['ratio'] >= 0.7) & (df1['ratio'] < 1.0)]['y_result']

# 시각화를 위해 데이터 묶기
plot_df = pd.DataFrame({
    "y_result": pd.concat([group1, group2, group3], ignore_index=True),
    "Group": (["[0,0.2)"] * len(group1)) +
             (["[0.2,0.7)"] * len(group2)) +
             (["[0.7,1)"] * len(group3))
})

# 박스플롯
plt.figure(figsize=(8,6))
sns.boxplot(x="Group", y="y_result", data=plot_df, palette="Set2")
sns.pointplot(x="Group", y="y_result", data=plot_df, estimator="mean",
              color="red", markers="D", linestyles="")
plt.title("그룹별 y_result 분포 (박스플롯)")
plt.show()

# 바플롯 (평균 + 신뢰구간)
plt.figure(figsize=(8,6))
sns.barplot(x="Group", y="y_result", data=plot_df, palette="Set2", ci=95)
plt.title("그룹별 평균 y_result (Barplot + 95% CI)")
plt.show()


## 사후검정 TUKEY HSD

df = df1.copy()
group1 = df[(df['ratio'] >= 0.0) & (df['ratio'] < 0.2)]['y_result']   # [0,0.2)
group2 = df[(df['ratio'] >= 0.2) & (df['ratio'] < 0.7)]['y_result']   # [0.2,0.7)
group3 = df[(df['ratio'] >= 0.7) & (df['ratio'] < 1.0)]['y_result']   # [0.7,1)

# --- (선택) 등분산성 체크: 등분산이 아니면 Tukey 대신 Games-Howell 권장 ---
lev_stat, lev_p = stats.levene(group1, group2, group3, center='median')
print(f"Levene 등분산성 p-value: {lev_p:.4g}")

# --- Tukey HSD 실행 ---
# 시각화/검정을 위해 하나의 시리즈로 묶고 그룹 라벨을 부여
y = pd.concat([group1, group2, group3], ignore_index=True)
g = (["[0,0.2)"] * len(group1) +
     ["[0.2,0.7)"] * len(group2) +
     ["[0.7,1)"] * len(group3))

tukey = pairwise_tukeyhsd(endog=y, groups=g, alpha=0.05)

print("\n[Tukey HSD 결과]")
print(tukey)  # 표 형태로 요약 출력

# 결과를 DataFrame으로 활용하고 싶다면
tukey_df = pd.DataFrame(
    tukey._results_table.data[1:],  # 헤더 제외
    columns=tukey._results_table.data[0]
)
print("\nDataFrame 형태 요약:")
print(tukey_df)

# 가설 3
시리즈물은 1편보다 후속작의 흥행 확률이 더 높을 것이다.

In [ ]:
# 시리즈물 뽑아내기

# ===== 시리즈물 추출 & 그룹핑 (하이브리드 규칙 + 쉼표 split 적용) =====
# 전제: df = pd.read_csv("TMDB_processed_final.csv") 완료됨
# 필요 컬럼: ['title','release_date','director','cast','keywords']

import pandas as pd
import numpy as np
import re
from collections import defaultdict
from itertools import combinations

df = pd.read_csv('/content/TMDB_processed_final.csv')


# -----------------------------
# 1) 유틸 함수
# -----------------------------
STOPWORDS = set("""a an and the of to in on at for from with by or as into about over under after before
part episode chapter pt vol volume saga story legend rise return revenge war age dawn day night vs versus
don t s a an the of and""".split())
ROMAN = {"i","ii","iii","iv","v","vi","vii","viii","ix","x"}

def normalize_title(t: str) -> str:
    if not isinstance(t, str):
        return ""
    t = t.lower().strip()
    t = re.sub(r"\s*\([^()]*\)\s*$", "", t)           # 끝 괄호 부제 제거
    t = re.sub(r"[^\w\s:\-]", " ", t)                 # 문장부호 정리(콜론/대시는 유지)
    t = re.sub(r"\s{2,}", " ", t).strip()
    return t

def tokenize(t: str):
    return [w for w in re.split(r"[\s:\-]+", t) if w]

def remove_leading_numbers(tokens):
    i = 0
    while i < len(tokens) and (tokens[i].isdigit() or tokens[i] in ROMAN):
        i += 1
    return tokens[i:]

def sig_tokens(tokens, max_k=None):
    sig = [w for w in tokens if (w not in STOPWORDS and not w.isdigit() and w not in ROMAN)]
    return sig[:max_k] if max_k else sig

def anchor2(title):
    """제목 전체에서 유의미한 앞 2토큰"""
    t = normalize_title(title)
    toks = remove_leading_numbers(tokenize(t))
    sig = sig_tokens(toks)
    return " ".join(sig[:2]) if len(sig) >= 2 else ""

def base3(title):
    """콜론/대시 앞의 첫 세그먼트에서 유의미한 앞 3토큰"""
    t = normalize_title(title)
    first_seg = re.split(r"[:\-]", t, maxsplit=1)[0].strip()
    toks = remove_leading_numbers(tokenize(first_seg))
    sig = sig_tokens(toks)
    return " ".join(sig[:3]) if sig else ""

def has_seq_suffix(title):
    """제목 끝의 숫자/로마숫자/Part/Chapter/Episode/Vol 패턴"""
    t = normalize_title(title)
    if re.search(r"(part|chapter|episode|pt|vol|volume)\s*(\d+|i|ii|iii|iv|v|vi|vii|viii|ix|x)\s*$", t):
        return True
    if re.search(r"\b(\d+|i|ii|iii|iv|v|vi|vii|viii|ix|x)\s*$", t):
        return True
    return False

def year_from_date(s):
    try:
        return int(str(s)[:4])
    except:
        return np.nan

# === 여기서 배우/키워드 split ===
def parse_csv_list(s, max_n=None, lower=True):
    """쉼표 기준 split → strip → lower → 리스트"""
    if not isinstance(s, str):
        return []
    items = [x.strip() for x in s.split(",") if x.strip()]
    if lower:
        items = [x.lower() for x in items]
    return items[:max_n] if max_n else items

def jacc(a, b):
    A, B = set(a), set(b)
    if not A or not B:
        return 0.0
    return len(A & B) / len(A | B)

# -----------------------------
# 2) 특징 생성
# -----------------------------
df = df.copy()
df['year'] = df['release_date'].apply(year_from_date)
df['anchor2'] = df['title'].fillna("").apply(anchor2)
df['base3']   = df['title'].fillna("").apply(base3)
df['seq_suffix'] = df['title'].fillna("").apply(has_seq_suffix)

# 배우: 상위 5명, lead3: 상위 3명
df['cast_list'] = df['cast'].apply(lambda s: parse_csv_list(s, max_n=5))
df['lead3']     = df['cast'].apply(lambda s: parse_csv_list(s, max_n=3))
# 키워드: 전체 사용
df['keywords_list'] = df['keywords'].apply(lambda s: parse_csv_list(s))

df.replace({"anchor2": {"": np.nan}, "base3": {"": np.nan}}, inplace=True)

# -----------------------------
# 3) 같은 anchor2 내에서 점수 계산
# -----------------------------
groups = defaultdict(list)
for idx, row in df.dropna(subset=['anchor2']).iterrows():
    groups[row['anchor2']].append(idx)

def qualify_pair(i, j):
    a = df.loc[i]; b = df.loc[j]
    if a['anchor2'] != b['anchor2']:
        return False, 0.0

    base_match   = (a['base3'] == b['base3'])
    same_director = isinstance(a['director'], str) and a['director'] != "" and a['director'] == b['director']
    cast_sim     = jacc(a['cast_list'], b['cast_list'])
    lead_overlap = len(set(a['lead3']) & set(b['lead3'])) >= 1
    keyw_sim     = jacc(a['keywords_list'], b['keywords_list'])
    ygap         = abs((a['year'] if pd.notna(a['year']) else 0) - (b['year'] if pd.notna(b['year']) else 0))
    has_sequel   = a['seq_suffix'] or b['seq_suffix']

    # 점수화
    score = 0.0
    if has_sequel:       score += 0.50
    if same_director:    score += 0.30
    if cast_sim >= 0.25: score += 0.25
    if lead_overlap:     score += 0.25
    if keyw_sim >= 0.50: score += 0.15
    if pd.notna(a['year']) and pd.notna(b['year']) and ygap <= 12: score += 0.10

    threshold = 0.80 if base_match else 0.95
    return score >= threshold, score

edges = []
for key, idxs in groups.items():
    if len(idxs) < 2:
        continue
    for i, j in combinations(idxs, 2):
        ok, _ = qualify_pair(i, j)
        if ok:
            edges.append((i, j))

# -----------------------------
# 4) Union-Find로 그룹핑
# -----------------------------
parent = {}
def find(x):
    parent.setdefault(x, x)
    if parent[x] != x:
        parent[x] = find(parent[x])
    return parent[x]
def union(a, b):
    ra, rb = find(a), find(b)
    if ra != rb:
        parent[rb] = ra

for i, j in edges:
    union(i, j)

comp = defaultdict(list)
for idx in set([k for e in edges for k in e]):
    comp[find(idx)].append(idx)

# -----------------------------
# 5) 결과 DataFrame
# -----------------------------
rows = []
for r, idxs in comp.items():
    titles = df.loc[idxs, 'title'].tolist()
    years  = df.loc[idxs, 'year'].dropna().astype(int).tolist()
    name   = df.loc[idxs, 'anchor2'].value_counts().idxmax()
    rows.append({
        "series_key": r,
        "series_name": name,
        "n_titles": len(idxs),
        "years_min": min(years) if years else np.nan,
        "years_max": max(years) if years else np.nan,
        "titles": sorted(titles)
    })

series_df = pd.DataFrame(rows).sort_values(["n_titles","series_name"], ascending=[False, True]).reset_index(drop=True)

# 원본 df에 라벨 부여
in_series_titles = set(t for r in rows for t in r["titles"])
df['is_series']   = df['title'].isin(in_series_titles)
name_map = {}
for r in rows:
    for t in r["titles"]:
        name_map[t] = r["series_name"]
df['series_group'] = df['title'].map(name_map)

# -----------------------------
# 6) 사용 예
# -----------------------------
print("감지된 시리즈 그룹 수:", len(series_df))
print(series_df.head(15)[["series_name","n_titles","years_min","years_max"]])

# series_df.to_csv("series_groups_final.csv", index=False)
# df.to_csv("TMDB_with_series_labels.csv", index=False)


# 추론 통계 코드

# -*- coding: utf-8 -*-
"""
TMDB_processed_final_with_series.csv 기반
'계층적 비교(1→2, 2→3, …; 중간 없으면 1→3처럼 다음 유효 편과 비교)'에 대해
부호검정(Sign test)과 윌콕슨 부호순위검정(Wilcoxon signed-rank)을 함께 수행하여
방향(누가 더 우세?)과 차이의 크기(효과 크기, 중앙값/평균)를 모두 보여주는 스크립트.

- y_result 컬럼은 CSV에 이미 존재한다고 가정 (0~3의 순서형 점수)
- Avengers(1998)는 avengers 시리즈에서 제외
- 결과 저장(현재 작업 폴더):
    1) pairs_level_details.csv      : 모든 시리즈의 (i→j) 페어 상세 및 diff
    2) level_tests_summary.csv      : 각 전이 레벨(예: 1→2, 2→3, 1→3 등)별 검정 요약
    3) overall_tests_summary.csv    : 전체 페어에 대한 검정 요약
"""

import os
import math
import numpy as np
import pandas as pd
from datetime import datetime

# SciPy가 있으면 정확 검정 사용, 없으면 대체 절차(정규근사/부호검정)로 동작
try:
    from scipy.stats import binomtest, wilcoxon
    SCIPY = True
except Exception:
    SCIPY = False

# ---------- (0) 파일 로드 ----------
PATH = "/content/TMDB_processed_final_with_series.csv"
assert os.path.exists(PATH), f"파일이 없습니다: {PATH}"
df = pd.read_csv(PATH)

# ---------- (1) 컬럼명 설정 (필요시 아래만 수정) ----------
title_col = "title"
series_col = "series_group"
release_date_col = "release_date"
yresult_col = "y_result"

# ---------- (2) 연도 추출(정렬용) ----------
df["year_extracted"] = pd.to_datetime(df[release_date_col], errors="coerce").dt.year

# ---------- (3) Avengers(1998) 시리즈 제외 ----------
mask_avengers_1998 = (
    df[title_col].astype(str).str.strip().str.lower().eq("the avengers")
    & (df["year_extracted"] == 1998)
    & df[series_col].astype(str).str.lower().str.contains("avengers", na=False)
)
df.loc[mask_avengers_1998, series_col] = np.nan
print(f"[INFO] 'The Avengers (1998)' 시리즈 그룹 제외 행 수: {int(mask_avengers_1998.sum())}")

# ---------- (4) 시리즈 내 정렬 및 installment 부여 ----------
series_df = df[df[series_col].notna()].copy()
series_df = series_df.sort_values(
    by=[series_col, "year_extracted", release_date_col, title_col],
    ascending=[True, True, True, True],
)
series_df["installment"] = series_df.groupby(series_col).cumcount() + 1
series_df[yresult_col] = pd.to_numeric(series_df[yresult_col], errors="coerce")

# ---------- (5) '가장 가까운 다음 편'과 짝짓기
# 정렬된 순서에서 인접 행끼리 묶기 → 중간 편이 비어 있으면 자연스럽게 1→3 같은 페어가 만들어짐
pairs = []
for grp, g in series_df.groupby(series_col, sort=False):
    g = g.sort_values("installment")
    rows = list(g.itertuples(index=False))
    for a, b in zip(rows[:-1], rows[1:]):   # 인접 행끼리 페어링
        i_inst = int(getattr(a, "installment"))
        j_inst = int(getattr(b, "installment"))
        i_y = getattr(a, yresult_col)
        j_y = getattr(b, yresult_col)
        pairs.append({
            "series_group": grp,
            "i_installment": i_inst,
            "j_installment": j_inst,
            "level_label": f"{i_inst}->{j_inst}",  # 계층적 전이 라벨
            "i_title": getattr(a, title_col),
            "j_title": getattr(b, title_col),
            "i_year": int(getattr(a, "year_extracted")) if pd.notna(getattr(a, "year_extracted")) else None,
            "j_year": int(getattr(b, "year_extracted")) if pd.notna(getattr(b, "year_extracted")) else None,
            "i_y_result": i_y,
            "j_y_result": j_y,
        })

pairs_df = pd.DataFrame(pairs).dropna(subset=["i_y_result", "j_y_result"])
pairs_df["i_y_result"] = pairs_df["i_y_result"].astype(int)
pairs_df["j_y_result"] = pairs_df["j_y_result"].astype(int)
pairs_df["diff"] = pairs_df["j_y_result"] - pairs_df["i_y_result"]
pairs_df["win"] = np.where(pairs_df["diff"] > 0, 1, np.where(pairs_df["diff"] < 0, -1, 0))

print(f"[INFO] 생성된 계층 페어 수: {len(pairs_df)}")
if len(pairs_df) == 0:
    raise SystemExit("[ERROR] 유효한 페어가 없습니다. 시리즈/컬럼 구성을 확인해 주세요.")

# ---------- (6) 검정 함수 ----------
def sign_test(win_series):
    """부호검정(Sign test)을 이항검정으로 수행: H1: 후속 승 비율 > 0.5"""
    s = win_series.dropna()
    s = s[s != 0]  # 동률/NA 제외
    n_plus = int((s == 1).sum())
    n_minus = int((s == -1).sum())
    n = n_plus + n_minus
    if n == 0:
        return {"n_eff": 0, "wins_sequel": n_plus, "wins_prior": n_minus, "p_one": np.nan, "p_two": np.nan}
    if SCIPY:
        p_two = binomtest(n_plus, n, 0.5, alternative="two-sided").pvalue
        p_one = binomtest(n_plus, n, 0.5, alternative="greater").pvalue
    else:
        # 정규근사(간이)
        phat = n_plus / n
        se = math.sqrt(0.25 / n)
        z = (phat - 0.5) / se
        p_one = 0.5 * (1 - math.erf(z / math.sqrt(2)))
        p_two = min(1.0, 2 * p_one)
    return {"n_eff": n, "wins_sequel": n_plus, "wins_prior": n_minus, "p_one": p_one, "p_two": p_two}

def wilcoxon_or_sign(diff_series):
    """Wilcoxon 한쪽/양쪽 p; SciPy 없으면 부호검정 대체"""
    d = diff_series.dropna()
    d = d[d != 0]
    if len(d) == 0:
        return {"n_eff": 0, "stat": np.nan, "p_one": np.nan, "p_two": np.nan}
    if SCIPY:
        stat_g, p_one = wilcoxon(d, alternative="greater", zero_method="wilcox")
        stat_t, p_two = wilcoxon(d, alternative="two-sided", zero_method="wilcox")
        return {"n_eff": int(len(d)), "stat": float(stat_g), "p_one": float(p_one), "p_two": float(p_two)}
    else:
        # 부호검정으로 대체
        n_plus = int((d > 0).sum()); n_minus = int((d < 0).sum()); n = n_plus + n_minus
        if n == 0:
            return {"n_eff": 0, "stat": np.nan, "p_one": np.nan, "p_two": np.nan}
        phat = n_plus / n
        se = math.sqrt(0.25 / n)
        z = (phat - 0.5) / se
        p_one = 0.5 * (1 - math.erf(z / math.sqrt(2)))
        p_two = min(1.0, 2 * p_one)
        return {"n_eff": n, "stat": np.nan, "p_one": p_one, "p_two": p_two}

def summarize_block(block_df, label="OVERALL"):
    """한 전이 레벨(예: 1→2) 또는 전체에 대해 Sign/Wilcoxon + 효과크기 요약"""
    wins = block_df["win"].fillna(0)
    sign_res = sign_test(wins)

    wres = wilcoxon_or_sign(block_df["diff"])

    # 효과 크기(차이의 크기) 요약: 중앙값/평균/표준편차, 양/음/0 개수
    diff = block_df["diff"]
    diff_nz = diff[diff != 0].dropna()
    summary = {
        "level_label": label,
        "pairs_total": int(len(block_df)),
        "pairs_used_sign": int(sign_res["n_eff"]),
        "pairs_used_wilcoxon": int(wres["n_eff"]),
        "wins_sequel": int(sign_res["wins_sequel"]),
        "wins_prior": int(sign_res["wins_prior"]),
        "ties_or_na": int((wins == 0).sum()),
        "diff_median": float(np.nanmedian(diff)) if len(diff) > 0 else np.nan,
        "diff_mean": float(np.nanmean(diff)) if len(diff) > 0 else np.nan,
        "diff_std": float(np.nanstd(diff, ddof=1)) if len(diff) > 1 else np.nan,
        "diff_pos_count": int((diff > 0).sum()),
        "diff_neg_count": int((diff < 0).sum()),
        "diff_zero_count": int((diff == 0).sum()),
        "sign_p_one_sided(H1: sequel > prior)": sign_res["p_one"],
        "sign_p_two_sided": sign_res["p_two"],
        "wilcoxon_stat": wres["stat"],
        "wilcoxon_p_one_sided(H1: sequel > prior)": wres["p_one"],
        "wilcoxon_p_two_sided": wres["p_two"],
    }
    return summary

# ---------- (7) 레벨별 요약 ----------
level_summaries = []
for lvl, g in pairs_df.groupby("level_label", sort=False):
    level_summaries.append(summarize_block(g, label=lvl))

level_summary_df = pd.DataFrame(level_summaries).sort_values(
    by=["level_label", "pairs_total"], ascending=[True, False]
)

# ---------- (8) 전체(OVERALL) 요약 ----------
overall_summary_df = pd.DataFrame([summarize_block(pairs_df, label="OVERALL")])

# ---------- (9) 저장 ----------
pairs_df.to_csv("pairs_level_details.csv", index=False)
level_summary_df.to_csv("level_tests_summary.csv", index=False)
overall_summary_df.to_csv("overall_tests_summary.csv", index=False)

print("[저장 완료] pairs_level_details.csv")
print("[저장 완료] level_tests_summary.csv")
print("[저장 완료] overall_tests_summary.csv")

# ---------- (10) 콘솔 표시(요약) ----------
print("\n=== OVERALL SUMMARY ===")
print(overall_summary_df.to_string(index=False))

print("\n=== LEVEL SUMMARY (head) ===")
print(level_summary_df.head(20).to_string(index=False))

# 가설 4
시대에 따라 흥행하는 장르가 다르다.(1990~2025년 *5년)

In [ ]:
#한글 글씨 폰트 설치
%%capture
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
fm.fontManager.addfont('/usr/share/fonts/truetype/nanum/NanumGothic.ttf')
plt.rcParams['font.family'] = 'NanumGothic'

# 표에서 ('-') 마이너스 표시
plt.rcParams['axes.unicode_minus'] = False

# 필요한 라이브러리 임포트
import pandas as pd # 데이터 분석 라이브러리
import numpy as np
import matplotlib.pyplot as plt # 시각화 도구 라이브러리1
import seaborn as sns # 시각화 도구 라이브러리2

df = pd.read_csv("/content/movies_genres_ohe.csv", on_bad_lines='skip')

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import shapiro, f_oneway, kruskal

df.columns = df.columns.str.strip()

df["release_date"] = pd.to_datetime(df["release_date"], errors="coerce")
df["year"] = df["release_date"].dt.year

df = df[(df["year"] >= 1990) & (df["year"] <= 2025)].copy()


bins   = list(range(1990, 2030, 5))
labels = [f"{y}–{y+4}" for y in bins[:-1]]
df["period"] = pd.cut(df["year"], bins=bins, labels=labels, right=True, include_lowest=True)


cols = df.loc[:, "Action":"Western"].columns.tolist()

col_counts = df[cols].sum().sort_values(ascending=False)
print("[1990~2025] 장르별 전체 편수:\n", col_counts, "\n")

period_genre = {}
for c in cols:
    period_genre[c] = df.loc[df[c] == 1].groupby("period")["y_result"].mean()

period_genre = pd.DataFrame(period_genre).sort_index()
print("5년 단위 × 장르별 평균 y_result (1990~2025):")
print(period_genre, "\n")


from scipy.stats import shapiro, levene, f_oneway, kruskal


print("[정규성 검정: Shapiro-Wilk] (1990~2025, 전체 장르)")
normality_results = {}
for c in cols:  # cols = ['Action', ..., 'Western']
    vals = df.loc[df[c] == 1, "y_result"].dropna()
    if len(vals) >= 8:
        stat, p = shapiro(vals)
        normality_results[c] = (stat, p, len(vals))
        print(f"{c:15s} → W={stat:.4f}, p={p:.4e}, n={len(vals)}")
    else:
        print(f"{c:15s} → 샘플 부족 (n={len(vals)})")

groups = [df.loc[df[c] == 1, "y_result"].dropna() for c in cols if len(df.loc[df[c] == 1, "y_result"].dropna()) > 1]
stat, p = levene(*groups)
print(f"\n[등분산성 검정: Levene]\nW={stat:.4f}, p={p:.4e}")

if all(pv > 0.05 for _, pv, _ in normality_results.values()):

    f_stat, p_val = f_oneway(*groups)
    print(f"\n[ANOVA 결과]\nF={f_stat:.4f}, p={p_val:.4e}")
else:
    h_stat, p_val = kruskal(*groups)
    print(f"\n[Kruskal-Wallis 결과]\nH={h_stat:.4f}, p={p_val:.4e}")

df["release_year"] = pd.to_datetime(df["release_date"], errors="coerce").dt.year
df = df[df["release_year"].notna()].copy()
df["release_year"] = df["release_year"].astype(int)

df["year_bin"] = (df["release_year"] // 5) * 5


df = df[df["year_bin"] >= 1990].copy()
df.loc[df["release_year"] >= 2025, "year_bin"] = 2025


cols = df.loc[:, "Action":"Western"].columns.tolist()

latest_bin = df["year_bin"].max()

df_latest = df[df["year_bin"] == latest_bin]


genre_mean_latest = {
    c: df_latest.loc[df_latest[c] == 1, "y_result"].mean() for c in cols
}
genre_mean_latest = pd.Series(genre_mean_latest).dropna().sort_values(ascending=False)

top5 = genre_mean_latest.head(5).index.tolist()
print(f" {latest_bin}년대 기준 Top5 장르:", top5)

trend_top5 = {c: df.loc[df[c] == 1].groupby("year_bin")["y_result"].mean() for c in top5}
trend_top5 = pd.DataFrame(trend_top5).sort_index()


trend_top5_pct = trend_top5.div(trend_top5.sum(axis=1).replace(0, np.nan), axis=0) * 100


ax = trend_top5_pct.plot(kind="bar", stacked=True, figsize=(14,7), alpha=0.9)
ax.set_title(f"최신구간 기준 1990~{latest_bin} (5년 단위) Top5 장르 y_result 비율 (100%)")
ax.set_xlabel("Year Bin (5년 단위)")
ax.set_ylabel("비율 (%)")
ax.legend(title="장르", bbox_to_anchor=(1.02, 1), loc="upper left")


for container in ax.containers:
    labels = [f"{h.get_height():.0f}%" if h.get_height() >= 8 else "" for h in container]
    ax.bar_label(container, labels=labels, label_type='center', fontsize=9)

plt.tight_layout()
plt.show()


import pandas as pd
import matplotlib.pyplot as plt

data = {
    "Year Bin": [1990, 1995, 2000, 2005, 2010, 2015, 2020],
    "Horror": [19, 16, 19, 18, 18, 20, 21],
    "Animation": [16, 26, 23, 25, 22, 24, 21],
    "War": [27, 18, 14, 16, 16, 14, 20],
    "Science Fiction": [17, 19, 20, 17, 21, 20, 19],
    "Family": [21, 21, 24, 24, 22, 22, 19]
}
df_plot = pd.DataFrame(data)

plt.figure(figsize=(12,6))
for genre in df_plot.columns[1:]:
    plt.plot(df_plot["Year Bin"], df_plot[genre], marker="o", label=genre)

    for x, y in zip(df_plot["Year Bin"], df_plot[genre]):
        plt.text(x, y+0.5, f"{y}%", ha="center", va="bottom", fontsize=9)

plt.title("최신구간 기준 1990~2020 (5년 단위) Top5 장르 y_result 비율 (선그래프)")
plt.xlabel("Year Bin (5년 단위)")
plt.ylabel("비율 (%)")
plt.legend(title="장르")
plt.grid(True, linestyle="--", alpha=0.5)
plt.tight_layout()
plt.show()




df["release_date"] = pd.to_datetime(df["release_date"], errors="coerce")
df["year"] = df["release_date"].dt.year
df = df[df["year"] >= 1980].copy()

bins = list(range(1980, 2025, 5))
labels = [f"{y}–{y+4}" for y in bins[:-1]]
df["five_year_bin"] = pd.cut(df["year"], bins=bins, labels=labels, right=True, include_lowest=True)

genre_cols = [
    'Action','Adventure','Animation','Comedy','Crime','Documentary','Drama','Family',
    'Fantasy','History','Horror','Music','Mystery','Romance','Science Fiction',
    'TV Movie','Thriller','War','Western'
]
genre_cols = [c for c in genre_cols if c in df.columns]

genre_by_period = df.groupby("five_year_bin")[genre_cols].sum()

print("=== 시대 × 장르별 영화 편수 ===")
print(genre_by_period)

df["release_date"] = pd.to_datetime(df["release_date"], errors="coerce")
df["year"] = df["release_date"].dt.year
df = df[df["year"] >= 1980].copy()

bins   = list(range(1980, 2025, 5))
labels = [f"{y}–{y+4}" for y in bins[:-1]]
df["five_year_bin"] = pd.cut(df["year"], bins=bins, labels=labels, right=True, include_lowest=True)


genre_cols = df.loc[:, "Action":"Western"].columns.tolist()

count_tbl = df.groupby("five_year_bin")[genre_cols].sum().reindex(labels).fillna(0).astype(int)


first_seen = {}
for g in genre_cols:
    nz = np.where(count_tbl[g].values > 0)[0]
    first_seen[g] = labels[nz[0]] if len(nz) else None

plt.figure(figsize=(14,8))
sns.heatmap(count_tbl.T, cmap="YlGnBu", linewidths=.5, linecolor="lightgray", cbar_kws={"label":"편수"})
plt.title("시대(5년) × 장르별 영화 편수")
plt.xlabel("5년 구간")
plt.ylabel("장르")

for gi, g in enumerate(count_tbl.columns):
    pass

for gi, g in enumerate(count_tbl.columns):
    pass


M = count_tbl.T
for gi, g in enumerate(M.index):
    nz = np.where(M.loc[g].values > 0)[0]
    if len(nz) > 0:
        fi = nz[0]
        plt.scatter(fi+0.5, gi+0.5, marker="*", s=120, color="red")

plt.tight_layout()
plt.show()

print("=== 장르별 최초 등장 구간 ===")
for g, f in first_seen.items():
    print(f"{g:15s}: {f}")


plt.figure(figsize=(12,7))
for col in count_tbl.columns:
    plt.plot(count_tbl.index, count_tbl[col], marker="o", label=col, alpha=0.7)

plt.title("시대별 모든 장르 영화 편수 추세")
plt.xlabel("5년 구간")
plt.ylabel("편수")
plt.xticks(rotation=45)
plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left")
plt.tight_layout()
plt.show()




def summarize_animation(df, start=1990, end=2025, step=5):
    df = df.copy()

    df["release_date"] = pd.to_datetime(df["release_date"], errors="coerce")
    df["year"] = df["release_date"].dt.year
    df = df[(df["year"] >= start) & (df["year"] <= end)].copy()

    bins   = list(range(start, end + step, step))
    labels = [f"{y}–{y+step-1}" for y in bins[:-1]]
    df["period"] = pd.cut(df["year"], bins=bins, labels=labels,
                          include_lowest=True, right=True)

    totals_cnt  = df.groupby("period", observed=False).size().reindex(labels).astype(float)
    totals_perf = df.groupby("period", observed=False)["y_result"].sum().reindex(labels).astype(float)

    ani_cnt  = df.groupby("period", observed=False)["Animation"].sum().reindex(labels).fillna(0.0)
    ani_perf = df.loc[df["Animation"]==1].groupby("period", observed=False)["y_result"] \
                 .sum().reindex(labels).fillna(0.0)
    ani_mean = df.loc[df["Animation"]==1].groupby("period", observed=False)["y_result"] \
                 .mean().reindex(labels)

    share_cnt  = ((ani_cnt  / totals_cnt.replace(0, np.nan))  * 100).fillna(0.0)
    share_perf = ((ani_perf / totals_perf.replace(0, np.nan)) * 100).fillna(0.0)

    excess_pp = (share_perf - share_cnt).fillna(0.0)

    summary = pd.DataFrame({
        "count": ani_cnt,
        "count_share_%": share_cnt,
        "perf_sum": ani_perf,
        "perf_share_%": share_perf,
        "mean_y_result": ani_mean,
        "excess_perf_pp": excess_pp
    }).reindex(labels).round(2)

    return summary, labels


anim_summary, labels = summarize_animation(df, start=1990, end=2025, step=5)
print(anim_summary)

plt.figure(figsize=(9,4))
plt.bar(anim_summary.index.astype(str), anim_summary["count"])
plt.title("Animation 제작 편수 (5년 단위)")
plt.xlabel("Period"); plt.ylabel("편수")
plt.xticks(rotation=45, ha="right")
plt.tight_layout(); plt.show()


plt.figure(figsize=(12,4))

plt.plot(anim_summary.index.astype(str), anim_summary["count_share_%"],
         marker="o", label="편수 점유율(%)", color="C0")
for x, y in zip(anim_summary.index.astype(str), anim_summary["count_share_%"]):
    plt.text(x, y-0.6, f"{y:.1f}%", ha="center", va="top", fontsize=9, color="C0")

plt.plot(anim_summary.index.astype(str), anim_summary["perf_share_%"],
         marker="o", label="성과 점유율(%)", color="C1")
for x, y in zip(anim_summary.index.astype(str), anim_summary["perf_share_%"]):
    plt.text(x, y+0.6, f"{y:.1f}%", ha="center", va="bottom", fontsize=9, color="C1")

plt.title("Animation 점유율(%) — 편수 vs 성과")
plt.xlabel("Period")
plt.ylabel("%")
plt.legend()
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()

plt.figure(figsize=(9,4))
plt.bar(anim_summary.index.astype(str), anim_summary["perf_sum"])
plt.title("Animation 성과 합 (y_result)")
plt.xlabel("Period"); plt.ylabel("합계 y_result")
plt.xticks(rotation=45, ha="right")
plt.tight_layout(); plt.show()

xs = np.arange(len(anim_summary))
y  = anim_summary["mean_y_result"].values
n  = anim_summary["count"].values

plt.figure(figsize=(9,4))
plt.plot(xs, y, marker="o")

for xi, yi, ni in zip(xs, y, n):
    if not np.isnan(yi):
        plt.text(xi, yi, f"n={int(ni)}", ha="center", va="bottom", fontsize=9)

plt.title("Animation 평균 y_result (5년 단위)")
plt.xlabel("Period"); plt.ylabel("평균 y_result")
plt.xticks(xs, anim_summary.index.astype(str), rotation=45, ha="right")
plt.tight_layout(); plt.show()

# 가설 5
장르별로 흥행 순위가 다를 것이다.

In [ ]:
## 추가 전처리

# 1. get_dummies()를 사용하여 장르를 별도의 열로 분리
# sep=', '는 쉼표와 공백을 기준으로 문자열을 나눈다는 의미입니다.
genre_dummies = df['genres'].str.get_dummies(sep=', ')

# 2. 기존 데이터프레임과 새로 생성된 장르 열을 합치기
df = pd.concat([df, genre_dummies], axis=1)
print("\n--- 장르를 열로 확장한 데이터프레임 ---")
print(df)

# CSV 파일로 저장
df.to_csv("/content/movies_genres_ohe.csv", index=False)
print("CSV 파일 저장 완료: movies_genres_ohe.csv")


## 장르별 평균 y_result 순위

# 1. 장르 컬럼만 자동으로 추출
non_genre_cols = [
    'id','title','vote_average','vote_count','status','release_date',
    'revenue','runtime','adult','budget','popularity','keywords','y_result', 'SR'
]
genre_cols = [c for c in df.columns if c not in non_genre_cols]

# 2. 각 장르별 평균 y_result 계산
genre_means = {}
for genre in genre_cols:
    mask = df[genre] == 1
    if mask.sum() > 0:
        genre_means[genre] = df.loc[mask, 'y_result'].mean()

# 3. 평균 y_result 높은 순으로 정렬
genre_means = pd.Series(genre_means).sort_values(ascending=False)

# 4. 결과 출력
print("장르별 평균 y_result (흥행 점수):")
print(genre_means)

# 5. 그래프로 시각화
plt.figure(figsize=(12,6))
genre_means.plot(kind='bar', color='skyblue')
plt.title('Average y_result by Genre', fontsize=14)
plt.ylabel('Average y_result', fontsize=12)
plt.xlabel('genre')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()


## 추론통계

groups = []
labels = []
for genre in genre_cols:
    mask = df[genre] == 1
    if mask.sum() > 0:
        groups.append(df.loc[mask, 'y_result'])
        labels.append(genre)

# 1. 정규성 검사 (Shapiro-Wilk)
print("=== Shapiro-Wilk normality test per group ===")
for label, data in zip(labels, groups):
    if len(data) >= 3:
        stat, p = stats.shapiro(data)
        print(f"{label:20s} W-stat={stat:.3f}, p={p:.3f}")
    else:
        print(f"{label:20s} (Too few samples for Shapiro test)")

# 2. 등분산성 검사 (Levene test)
stat, p = stats.levene(*groups)
print("\n=== Levene’s test for equal variances ===")
print(f"Levene stat={stat:.3f}, p={p:.3f}")

h_stat, p_val = stats.kruskal(*groups)

print("Kruskal–Wallis H-statistic:", h_stat)
print("p-value:", p_val)

# 3. Dunn's test

!pip install scikit-posthocs

import scikit_posthocs as sp
import pandas as pd

df_long = []

for genre in genre_cols:
    mask = df[genre] == 1
    if mask.sum() > 0:
        temp = df.loc[mask, ['y_result']].copy()
        temp['genre'] = genre
        df_long.append(temp)

df_long = pd.concat(df_long, axis=0)

# 4. Dunn’s test 수행 (p-value Holm 보정)
dunn_results = sp.posthoc_dunn(df_long, val_col='y_result', group_col='genre', p_adjust='holm')

print(dunn_results)


## 장르 조합별 평균 y_result 순위

# 장르 컬럼들
non_genre_cols = [
    'id','title','vote_average','vote_count','status','release_date',
    'revenue','runtime','adult','budget','popularity','keywords','y_result', 'SR'
]
genre_cols = [c for c in df.columns if c not in non_genre_cols]

# 장르 컬럼을 전부 0/1 정수로 변환
for col in genre_cols:
    df[col] = df[col].apply(lambda x: 1 if str(x).strip().lower() in ['1','true','yes'] else 0).astype(int)

# 장르 조합 컬럼 생성
from itertools import combinations

for genre1, genre2 in combinations(genre_cols, 2):
    combo_name = f"{genre1}_{genre2}"
    df[combo_name] = df[genre1] * df[genre2]

# 각 조합별 y_result 평균 계산
results = []
for genre1, genre2 in combinations(genre_cols, 2):
    combo_name = f"{genre1}_{genre2}"
    subset = df[df[combo_name] == 1]
    if len(subset) > 0:
        avg_y = subset['y_result'].mean()
        results.append({
            'Combination': combo_name,
            'NumMovies': len(subset),
            'Avg_y_result': avg_y
        })

combo_df = pd.DataFrame(results).sort_values('Avg_y_result', ascending=False)
print(combo_df.head(20))

# 장르1 & 장르2 조합별 Top10 그래프
top10 = combo_df.head(10)

plt.figure(figsize=(12,6))
sns.barplot(x='Avg_y_result', y='Combination', data=top10, palette='viridis')
plt.title('Top 10 Genre and Combination Coefficients on Box Office')
plt.xlabel('Coefficient')
plt.ylabel('Genre / Genre Combination')
plt.tight_layout()
plt.show()


# 가설 6
개봉 시기에 따른 흥행여부는 차이가 있을 것이다.

In [ ]:
# 1. 'genres' 열의 문자열을 쉼표 기준으로 잘라 리스트로 변환
#    이 결과를 바로 'genres_type'이라는 새 칼럼에 저장합니다.
df['genres_type'] = df['genres'].str.split(', ')


# 2. 새로 만든 'genres_type' 칼럼을 기준으로 explode를 적용
#    이제 df의 행들이 'genres_type' 리스트의 각 요소에 따라 확장됩니다.
df = df.explode('genres_type')


df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')
# 2. 날짜 형식으로 변환된 컬럼에서 '월'과 '분기' 정보 추출
# .dt 접근자를 사용하면 날짜 관련 정보를 쉽게 뽑아낼 수 있습니다.
df['month'] = df['release_date'].dt.month
df['quarter'] = df['release_date'].dt.quarter


import pandas as pd
from scipy.stats import chi2_contingency

# --- '월(month)'과 'Y Result' 간의 카이제곱 검정 ---

print("--- '월(month)' 기준 카이제곱 검정 ---")

# 1. 교차표(Contingency Table) 생성
# 행(index)에는 흥행 등급, 열(column)에는 개봉 월을 배치합니다.
contingency_table_month = pd.crosstab(df['y_result'], df['month'])

print("\n[월별 흥행 등급 교차표]")
print(contingency_table_month)

# 2. 카이제곱 검정 실행
# 교차표를 chi2_contingency 함수에 전달합니다.
chi2, p_value, dof, expected = chi2_contingency(contingency_table_month)

print(f"\n카이제곱 통계량(Chi-squared Statistic): {chi2:.4f}")
print(f"p-value: {p_value:.4f}")
print(f"자유도(Degrees of Freedom): {dof}")

# 3. 결과 해석
alpha = 0.05  # 유의수준 5%
if p_value < alpha:
    print("\n[결론] p-value가 유의수준보다 작으므로, '개봉 월'과 '흥행'은 통계적으로 유의미한 관련이 있습니다.")
else:
    print("\n[결론] p-value가 유의수준보다 크므로, '개봉 월'과 '흥행'의 관련성을 통계적으로 입증하지 못했습니다.")



import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency

# 이전에 생성한 교차표 'contingency_table_month'가 있다고 가정합니다.
# contingency_table_month = pd.crosstab(df['Y Result'], df['month'])

# 1. 카이제곱 검정 재실행하여 '기대 빈도' 값 얻기
chi2, p, dof, expected = chi2_contingency(contingency_table_month)

# 2. 표준화 잔차 계산
# 공식: (관측 빈도 - 기대 빈도) / sqrt(기대 빈도)
residuals = (contingency_table_month - expected) / np.sqrt(expected)

# 더 정확한 분석을 위해 조정 잔차(Adjusted Residuals)를 사용하는 경우도 많지만,
# 해석의 편의성을 위해 표준화 잔차로도 충분히 의미있는 결과를 얻을 수 있습니다.
# 여기서는 표준화 잔차(residuals)를 사용하겠습니다.

print("--- 표준화 잔차 (Standardized Residuals) ---")
print(residuals)


# 3. (선택사항) 결과를 보기 좋게 시각화하기 (강력 추천!)
# 잔차 값이 1.96보다 크거나 -1.96보다 작은 셀을 하이라이트하여 보여줍니다.
def highlight_significant_cells(val):
    if val > 1.96:
        return 'background-color: lightgreen' # 유의미하게 많음 (긍정적)
    elif val < -1.96:
        return 'background-color: lightcoral'  # 유의미하게 적음 (부정적)
    else:
        return ''

styled_residuals = residuals.style.applymap(highlight_significant_cells)

print("\n\n--- 유의미한 셀 하이라이트 ---")
# Jupyter Notebook이나 Colab 환경에서 실행하면 색상이 입혀진 표가 보입니다.
display(styled_residuals)


# 1. '성공 그룹' 데이터 필터링 (y_result가 2 또는 3)
success_df = df[df['y_result'].isin([2, 3])].copy()

# 2. 예산이 0보다 크고, 결측치가 없는 데이터만 남기기
success_df = success_df[success_df['budget'] > 0].dropna()

print("성공 그룹 데이터 준비 완료!")
print(f"분석에 사용할 성공 영화 개수: {len(success_df)}")


from scipy.stats import ttest_ind

# 2단계에서 생성한 success_df 사용

# 1. '성수기'와 '비수기' 그룹으로 데이터 나누기
peak_season_months = [5, 6, 7, 11, 12]
peak_group = success_df[success_df['month'].isin(peak_season_months)]['budget']
off_peak_group = success_df[~success_df['month'].isin(peak_season_months)]['budget']

print(f"성수기 성공작 평균 예산: ${peak_group.mean():,.2f}")
print(f"비수기 성공작 평균 예산: ${off_peak_group.mean():,.2f}")

# 2. T-test 실행
ttest_result = ttest_ind(peak_group, off_peak_group, equal_var=False)
print(f"\nT-test p-value: {ttest_result.pvalue:.10f}")

# 3. 결과 해석
if ttest_result.pvalue < 0.05:
    print(">> 결론: 성수기와 비수기 성공작들의 평균 예산 차이는 통계적으로 유의미합니다.")
else:
    print(">> 결론: 두 그룹의 평균 예산 차이는 통계적으로 유의미하지 않습니다.")

# 머신러닝 - 영화 예측 프로그램
전처리과정

In [ ]:
# ==============================================================================
# 0. 라이브러리
# ==============================================================================
import pandas as pd
import numpy as np
import re
from pathlib import Path
from scipy import sparse
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from imblearn.over_sampling import SMOTE
from sklearn.feature_extraction.text import TfidfVectorizer

# ==============================================================================
# 1. 데이터 로드 및 전처리
# ==============================================================================
CSV_PATH = "/content/TMDB_processed_final.csv"
TARGET = "y_result"

df = pd.read_csv(Path(CSV_PATH))

# 타겟 0,1,2 -> 0, 3 -> 1
y = df[TARGET].map(lambda v: 0 if v in [0,1] else 1).astype(int)
X = df.drop(columns=[TARGET])

# 날짜/수치형 파생
X['release_date'] = pd.to_datetime(X['release_date'], errors='coerce')
X['release_year'] = X['release_date'].dt.year
X['release_month'] = X['release_date'].dt.month
X['log_budget'] = np.log1p(X['budget'].clip(lower=0))

# 텍스트 정제
TEXT_COLS = ["keywords", "overview", "tagline"]
def clean_text(s):
    if pd.isna(s): return ""
    s = str(s).lower().strip()
    s = re.sub(r"\s*,\s*", " ", s)
    s = re.sub(r"\s+", " ", s)
    return s

for c in TEXT_COLS:
    X[c] = X[c].apply(clean_text)

# 텍스트 파생 피처
for col in TEXT_COLS:
    X[col + "_len"] = X[col].apply(lambda x: len(x))
    X[col + "_word_count"] = X[col].apply(lambda x: len(x.split()))

# 장르/감독 평균 점수 피처
genre_avg = df.groupby("genres")[TARGET].mean()
director_avg = df.groupby("director")[TARGET].mean()
X["genre_avg_score"] = X["genres"].map(genre_avg).fillna(genre_avg.mean())
X["director_avg_score"] = X["director"].map(director_avg).fillna(director_avg.mean())

# 날짜 파생
X['release_quarter'] = X['release_date'].dt.quarter.fillna(0)
X['is_recent'] = X['release_year'].apply(lambda x: 1 if x >= 2015 else 0)

# ==============================================================================
# 2. 학습/검증 분할
# ==============================================================================
RANDOM_STATE = 42
TEST_SIZE = 0.2
X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE, stratify=y
)

# ==============================================================================
# 3. TF-IDF 벡터화
# ==============================================================================
TFIDF_MAX_FEATURES = 20000
X_train['text_combined'] = X_train[TEXT_COLS].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
X_valid['text_combined'] = X_valid[TEXT_COLS].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)

vectorizer = TfidfVectorizer(max_features=TFIDF_MAX_FEATURES, token_pattern=r"[^ ]+")
X_train_tfidf = vectorizer.fit_transform(X_train['text_combined'])
X_valid_tfidf = vectorizer.transform(X_valid['text_combined'])

# ==============================================================================
# 4. 수치형 + 파생 피처
# ==============================================================================
num_feature_cols = ['runtime','log_budget','release_year','release_month',
                    'keywords_len','keywords_word_count',
                    'overview_len','overview_word_count',
                    'tagline_len','tagline_word_count',
                    'genre_avg_score','director_avg_score',
                    'release_quarter','is_recent']

X_train_num = X_train[num_feature_cols].fillna(0)
X_valid_num = X_valid[num_feature_cols].fillna(0)

# ==============================================================================
# 5. 최종 학습용 matrix 결합
# ==============================================================================
X_train_final = sparse.hstack([X_train_tfidf, sparse.csr_matrix(X_train_num.values)]).tocsr()
X_valid_final = sparse.hstack([X_valid_tfidf, sparse.csr_matrix(X_valid_num.values)]).tocsr()

# ==============================================================================
# 6. SMOTE 적용
# ==============================================================================
smote = SMOTE(random_state=RANDOM_STATE)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_final, y_train)

모델 학습

In [ ]:
# ==============================================================================
# 7. LightGBM 학습 (최적화 버전)
# ==============================================================================
lgb_model = lgb.LGBMClassifier(
    boosting_type='gbdt',
    objective='binary',
    learning_rate=0.05,
    n_estimators=500,
    num_leaves=31,
    max_depth=6,
    feature_fraction=0.9,
    bagging_fraction=0.8,
    bagging_freq=5,
    class_weight='balanced',
    random_state=RANDOM_STATE,
    n_jobs=-1
)

callbacks = [lgb.early_stopping(stopping_rounds=50), lgb.log_evaluation(100)]

lgb_model.fit(
    X_train_resampled, y_train_resampled,
    eval_set=[(X_valid_final, y_valid)],
    eval_metric='binary_logloss',
    callbacks=callbacks
)

# ==============================================================================
# 8. 평가
# ==============================================================================
y_pred = lgb_model.predict(X_valid_final)

print("\nAccuracy:", accuracy_score(y_valid, y_pred))
print("Precision:", precision_score(y_valid, y_pred))
print("Recall:", recall_score(y_valid, y_pred))
print("F1 Score:", f1_score(y_valid, y_pred))
print("\nClassification Report:\n", classification_report(y_valid, y_pred))